# Initializing needed libraries

In [74]:
import random
import time
import csv

# Problem 1




In [56]:
random.seed(42)  # Ensuring reproducibility
R1 = [(random.randint(1, 5), random.randint(1, 5)) for _ in range(10)]
R2 = [(random.randint(1, 5), random.randint(1, 5)) for _ in range(10)]

# Create hashmap for R2 with 'B' as the key
hashmap_R2 = {}
for b, c in R2:
    if b not in hashmap_R2:
        hashmap_R2[b] = []
    hashmap_R2[b].append(c)  # Store 'c' under key 'b'
# printing R1, R2 and Hashmap
print("R1 (A, B):", R1)
print("R2 (B, C):", R2)
print("Hashmap of R2 indexed by B:", hashmap_R2)

# Join R1 and R2 by B and print results
print("\nJoin Results:")
for a, b in R1:
    if b in hashmap_R2:
        for c in hashmap_R2[b]:
            print(f"({a}, {b}, {c})")


R1 (A, B): [(1, 1), (3, 2), (2, 2), (1, 5), (1, 5), (4, 1), (1, 1), (2, 2), (5, 5), (1, 5)]
R2 (B, C): [(2, 5), (4, 2), (4, 5), (3, 1), (2, 4), (3, 3), (2, 2), (3, 1), (1, 4), (1, 3)]
Hashmap of R2 indexed by B: {2: [5, 4, 2], 4: [2, 5], 3: [1, 3, 1], 1: [4, 3]}

Join Results:
(1, 1, 4)
(1, 1, 3)
(3, 2, 5)
(3, 2, 4)
(3, 2, 2)
(2, 2, 5)
(2, 2, 4)
(2, 2, 2)
(4, 1, 4)
(4, 1, 3)
(1, 1, 4)
(1, 1, 3)
(2, 2, 5)
(2, 2, 4)
(2, 2, 2)


# Problem 2

In [57]:
# Define the relations
R1 = {('1', '2'), ('4', '6'), ('7', '2')}
R2 = {('2', '21'), ('6', '5'), ('9', '3')}
R3 = {('21', '8'), ('5', '13'), ('3', '19')}

# Map relation names to actual relations
Relations = {
    ('A1', 'A2'): R1,
    ('A2', 'A3'): R2,
    ('A3', 'A4'): R3,
}

# Define the queries
queries = [(('A1', 'A2'), ('A2', 'A3'), ('A3', 'A4'))]

# Function to dynamically map relation keys to node labels based on the query
def map_relations_to_nodes(relations, query):
    node_mapping = {}
    index = 1
    for relation_key in query:
        if relation_key in relations:
            node_mapping[f"R{index}"] = relations[relation_key]
            index += 1
    return node_mapping

# Dynamically map relations based on the query
relations = map_relations_to_nodes(Relations, queries[0])
print("Mapped Relations:", relations)

# Function to create join tree dynamically based on the filtered relations
def create_join_tree_from_relations(relations):
    tree = {}
    nodes = list(relations.keys())
    for i, node in enumerate(nodes):
        next_node = nodes[i + 1] if i + 1 < len(nodes) else None
        tree[node] = [next_node] if next_node else []
    return tree

# Create join tree from filtered relations
join_tree = create_join_tree_from_relations(relations)
print("Generated Join Tree:", join_tree)

# Post-Order Traversal with Records
def post_order_traversal_with_records(tree, node, relations, visited=None):
    if visited is None:
        visited = []
    for child in tree[node]:
        if child:
            post_order_traversal_with_records(tree, child, relations, visited)
    visited.append(node)
    print(f"Visited {node}, Associated Records: {relations[node]}")
    return visited

# Get post-order traversal and print associated records
post_order = post_order_traversal_with_records(join_tree, 'R1', relations)
print("Post-order Traversal:", post_order)

# Bottom-Up Semi-Join Reduction
def bottom_up_semi_join(tree, post_order, relations):
    def semi_join_filter(R, index, values):
        return {t for t in R if t[index] in values}

    def get_common_indices(parent, child):
        print(f"Getting common indices between {parent} and {child}")
        if parent == 'R1' and child == 'R2':
            return 1, 0
        if parent == 'R2' and child == 'R3':
            return 1, 0
        raise ValueError(f"No common attribute found between {parent} and {child}")

    # Traverse from bottom to top (leaf to root)
    for node in reversed(post_order):
        if not tree[node]:
            continue
        for child in tree[node]:
            parent_idx, child_idx = get_common_indices(node, child)
            child_values = {t[child_idx] for t in relations[child]}
            relations[node] = semi_join_filter(relations[node], parent_idx, child_values)

    return relations

# Apply Bottom-Up Semi-Join Reduction
relations = bottom_up_semi_join(join_tree, post_order, relations)
print("Relations After Bottom-Up Semi-Join:")
for node in post_order:
    print(f"{node}: {relations[node]}")

# Top-Down Semi-Join Reduction
def top_down_semi_join(tree, post_order, relations):
    def semi_join_filter(R, index, values):
        return {t for t in R if t[index] in values}

    def get_common_indices(parent, child):
        print(f"Getting common indices between {parent} and {child}")
        if parent == 'R1' and child == 'R2':
            return 1, 0
        if parent == 'R2' and child == 'R3':
            return 1, 0
        raise ValueError(f"No common attribute found between {parent} and {child}")

    # Traverse from top to bottom (root to leaf)
    for node in post_order[::-1]:  # Reversed traversal
        for child in tree[node]:
            parent_idx, child_idx = get_common_indices(node, child)
            parent_values = {t[parent_idx] for t in relations[node]}
            relations[child] = semi_join_filter(relations[child], child_idx, parent_values)

    return relations

# Apply Top-Down Semi-Join Reduction
relations = top_down_semi_join(join_tree, post_order, relations)
print("Relations After Top-Down Semi-Join:")
for node in post_order:
    print(f"{node}: {relations[node]}")

# Final Join and Project Function
def final_join_and_project(relations, join_tree, post_order):
    def get_common_indices(parent, child):
        print(f"Getting common indices between {parent} and {child}")
        if parent == 'R1' and child == 'R2':
            return 1, 0  # Common attribute is at index 1 in R1, index 0 in R2
        if parent == 'R2' and child == 'R3':
            return 2, 0  # Updated: Common attribute is at index 2 in the join result and index 0 in R3
        raise ValueError(f"No common attribute found between {parent} and {child}")

    root_relation = relations['R1']
    joined_results = set()

    # Traverse from root to leaf nodes
    for parent in post_order[::-1]:  # Traverse from root to leaf
        children = join_tree[parent]
        for child in children:
            parent_idx, child_idx = get_common_indices(parent, child)
            print(f"Joining {parent} on index {parent_idx} with {child} on index {child_idx}")
            temp_results = set()
            for p in root_relation:
                for c in relations[child]:
                    print(f"Parent Tuple: {p}, Child Tuple: {c}")  # Debug print to see tuples
                    if p[parent_idx] == c[child_idx]:
                        temp_results.add(p + c[1:])
            root_relation = temp_results  # Update root_relation with the intermediate results
            print(f"Join with {child}: {root_relation}")  # Debug print

    return root_relation

# Apply final join and projection
final_result = final_join_and_project(relations, join_tree, post_order)
print("Final Joined Relations:")
for record in final_result:
  print(record)

Mapped Relations: {'R1': {('4', '6'), ('7', '2'), ('1', '2')}, 'R2': {('6', '5'), ('2', '21'), ('9', '3')}, 'R3': {('21', '8'), ('5', '13'), ('3', '19')}}
Generated Join Tree: {'R1': ['R2'], 'R2': ['R3'], 'R3': []}
Visited R3, Associated Records: {('21', '8'), ('5', '13'), ('3', '19')}
Visited R2, Associated Records: {('6', '5'), ('2', '21'), ('9', '3')}
Visited R1, Associated Records: {('4', '6'), ('7', '2'), ('1', '2')}
Post-order Traversal: ['R3', 'R2', 'R1']
Getting common indices between R1 and R2
Getting common indices between R2 and R3
Relations After Bottom-Up Semi-Join:
R3: {('21', '8'), ('5', '13'), ('3', '19')}
R2: {('9', '3'), ('6', '5'), ('2', '21')}
R1: {('4', '6'), ('7', '2'), ('1', '2')}
Getting common indices between R1 and R2
Getting common indices between R2 and R3
Relations After Top-Down Semi-Join:
R3: {('21', '8'), ('5', '13')}
R2: {('6', '5'), ('2', '21')}
R1: {('4', '6'), ('7', '2'), ('1', '2')}
Getting common indices between R1 and R2
Joining R1 on index 1 with

# Problem 3

In [61]:
# Generate synthetic data for relations
def generate_relation(num_tuples, attribute_range):
    return [(random.randint(1, attribute_range), random.randint(1, attribute_range)) for _ in range(num_tuples)]

# Hash join function
def hash_join(R1, R2, R1_join_idx, R2_join_idx):
    # Create hash table for R2
    hash_table = {}
    for record in R2:
        key = record[R2_join_idx]
        if key not in hash_table:
            hash_table[key] = []
        hash_table[key].append(record)

    # Execute the join
    join_result = []
    for record in R1:
        key = record[R1_join_idx]
        if key in hash_table:  # Checking if key exists in the hash table
            for matching_record in hash_table[key]:
                # Concatenate tuples excluding the join key from the second relation
                join_result.append(record + matching_record[1:])

    return join_result

def seq_join(relations,k):
  # Perform iterative joins
  result = relations[0]  # Start with the first relation
  for i in range(1, k):
    result = hash_join(result, relations[i], i, 0)  # Join on the last attribute of result and the first of the next relation
  return result

In [62]:

# Generate synthetic data for relations
def generate_relation(num_tuples, attribute_range):
    return [(random.randint(1, attribute_range), random.randint(1, attribute_range)) for _ in range(num_tuples)]
# Number of relations
k = 5  # Example with 5 relations for simplicity
attribute_range = 5  # Attribute values range from 1 to 5
num_tuples = 10  # Each relation has 10 tuples

# Generate relations
relations = [generate_relation(num_tuples, attribute_range) for _ in range(k)]


In [64]:
# Output the final join result
print("Final join result:", seq_join(relations,k))
len(seq_join(relations,k))

Final join result: [(5, 2, 2, 1, 1, 4), (5, 2, 2, 1, 1, 1), (5, 2, 2, 1, 1, 4), (5, 2, 2, 1, 3, 4), (5, 2, 2, 1, 3, 1), (5, 2, 2, 5, 2, 1), (5, 2, 2, 5, 2, 2), (5, 2, 2, 5, 2, 5), (5, 2, 2, 1, 1, 4), (5, 2, 2, 1, 1, 1), (5, 2, 2, 1, 1, 4), (5, 2, 2, 1, 3, 4), (5, 2, 2, 1, 3, 1), (5, 2, 3, 4, 2, 1), (5, 2, 3, 4, 2, 2), (5, 2, 3, 4, 2, 5), (5, 2, 3, 4, 2, 1), (5, 2, 3, 4, 2, 2), (5, 2, 3, 4, 2, 5), (5, 2, 3, 4, 4, 4), (5, 2, 3, 4, 4, 2), (5, 2, 3, 4, 2, 1), (5, 2, 3, 4, 2, 2), (5, 2, 3, 4, 2, 5), (5, 2, 3, 4, 2, 1), (5, 2, 3, 4, 2, 2), (5, 2, 3, 4, 2, 5), (5, 2, 3, 4, 4, 4), (5, 2, 3, 4, 4, 2), (5, 2, 3, 1, 1, 4), (5, 2, 3, 1, 1, 1), (5, 2, 3, 1, 1, 4), (5, 2, 3, 1, 3, 4), (5, 2, 3, 1, 3, 1), (5, 1, 2, 1, 1, 4), (5, 1, 2, 1, 1, 1), (5, 1, 2, 1, 1, 4), (5, 1, 2, 1, 3, 4), (5, 1, 2, 1, 3, 1), (5, 1, 2, 5, 2, 1), (5, 1, 2, 5, 2, 2), (5, 1, 2, 5, 2, 5), (5, 1, 2, 1, 1, 4), (5, 1, 2, 1, 1, 1), (5, 1, 2, 1, 1, 4), (5, 1, 2, 1, 3, 4), (5, 1, 2, 1, 3, 1), (5, 1, 5, 2, 1, 4), (5, 1, 5, 2, 1, 1), 

236

# Problem 4

In [99]:
def generate_datasets():
    R1 = [(i, random.randint(1, 5000)) for i in range(1, 101)]  # (i, x)
    R2 = [(random.randint(1, 5000), j) for j in range(1, 101)]  # (y, j)
    R3 = [(l, l) for l in range(1, 101)]  # (l, l)
    return R1, R2, R3

In [100]:
# Generate the datasets
R1, R2, R3 = generate_datasets()
relation3=[R1,R2,R3]
print("R1 Dataset: ",R1)
print("R2 Dataset: ",R2)
print("R3 Dataset: ",R3)

R1 Dataset:  [(1, 2404), (2, 4392), (3, 3029), (4, 1847), (5, 3101), (6, 1121), (7, 3425), (8, 4583), (9, 4652), (10, 340), (11, 2301), (12, 3831), (13, 1417), (14, 1621), (15, 4225), (16, 3555), (17, 2181), (18, 518), (19, 100), (20, 27), (21, 4880), (22, 2777), (23, 1861), (24, 4104), (25, 2763), (26, 2564), (27, 3947), (28, 2195), (29, 1429), (30, 749), (31, 3202), (32, 2282), (33, 1921), (34, 2632), (35, 4834), (36, 4944), (37, 132), (38, 267), (39, 4765), (40, 1806), (41, 3908), (42, 4372), (43, 637), (44, 2146), (45, 3501), (46, 409), (47, 2497), (48, 4004), (49, 3364), (50, 769), (51, 2406), (52, 1820), (53, 3733), (54, 4139), (55, 4281), (56, 2146), (57, 2260), (58, 1739), (59, 3745), (60, 420), (61, 3246), (62, 3074), (63, 675), (64, 4442), (65, 3698), (66, 2879), (67, 3405), (68, 2101), (69, 3393), (70, 3829), (71, 1077), (72, 3177), (73, 1916), (74, 3311), (75, 4534), (76, 551), (77, 3013), (78, 2433), (79, 714), (80, 4256), (81, 3054), (82, 4240), (83, 4989), (84, 4680), (8

Implementing Problem-2 Algorithm with above generated dataset

In [101]:
# Function to map relation keys to nodes based on query
def map_relations_to_nodes(relations, query):
    node_mapping = {}
    index = 1
    for relation_key in query:
        if relation_key in relations:
            node_mapping[f"R{index}"] = relations[relation_key]
            index += 1
    return node_mapping

# Function to create a join tree from relations
def create_join_tree_from_relations(relations):
    tree = {}
    nodes = list(relations.keys())
    for i, node in enumerate(nodes):
        next_node = nodes[i + 1] if i + 1 < len(nodes) else None
        tree[node] = [next_node] if next_node else []
    return tree

# Function to perform post-order traversal
def post_order_traversal_with_records(tree, node, relations, visited=None):
    if visited is None:
        visited = []
    for child in tree[node]:
        if child:
            post_order_traversal_with_records(tree, child, relations, visited)
    visited.append(node)
    return visited

# Function for bottom-up semi-join reduction
def bottom_up_semi_join(tree, post_order, relations):
    def semi_join_filter(R, index, values):
        return {t for t in R if t[index] in values}

    def get_common_indices(parent, child):
        if parent == 'R1' and child == 'R2':
            return 1, 0
        if parent == 'R2' and child == 'R3':
            return 1, 0
        raise ValueError(f"No common attribute found between {parent} and {child}")

    for node in reversed(post_order):
        if not tree[node]:
            continue
        for child in tree[node]:
            parent_idx, child_idx = get_common_indices(node, child)
            child_values = {t[child_idx] for t in relations[child]}
            relations[node] = semi_join_filter(relations[node], parent_idx, child_values)

    return relations

# Function for top-down semi-join reduction
def top_down_semi_join(tree, post_order, relations):
    def semi_join_filter(R, index, values):
        return {t for t in R if t[index] in values}

    def get_common_indices(parent, child):
        if parent == 'R1' and child == 'R2':
            return 1, 0
        if parent == 'R2' and child == 'R3':
            return 1, 0
        raise ValueError(f"No common attribute found between {parent} and {child}")

    for node in post_order[::-1]:  # Reversed traversal
        for child in tree[node]:
            parent_idx, child_idx = get_common_indices(node, child)
            parent_values = {t[parent_idx] for t in relations[node]}
            relations[child] = semi_join_filter(relations[child], child_idx, parent_values)

    return relations

# Function to perform final join and project
def final_join_and_project(relations, join_tree, post_order):
    def get_common_indices(parent, child):
        if parent == 'R1' and child == 'R2':
            return 1, 0
        if parent == 'R2' and child == 'R3':
            return 2, 0
        raise ValueError(f"No common attribute found between {parent} and {child}")

    root_relation = relations['R1']
    joined_results = set()

    for parent in post_order[::-1]:  # Traverse from root to leaf
        children = join_tree[parent]
        for child in children:
            parent_idx, child_idx = get_common_indices(parent, child)
            temp_results = set()
            for p in root_relation:
                for c in relations[child]:
                    if p[parent_idx] == c[child_idx]:
                        temp_results.add(p + c[1:])
            root_relation = temp_results  # Update root_relation with the intermediate results

    return root_relation

# Function to execute the whole operation with externally passed relations
def execute_operation(R1, R2, R3):
    Relations = {
        ('A1', 'A2'): R1,
        ('A2', 'A3'): R2,
        ('A3', 'A4'): R3,
    }
    queries = [(('A1', 'A2'), ('A2', 'A3'), ('A3', 'A4'))]

    relations = map_relations_to_nodes(Relations, queries[0])
    join_tree = create_join_tree_from_relations(relations)
    post_order = post_order_traversal_with_records(join_tree, 'R1', relations)

    relations = bottom_up_semi_join(join_tree, post_order, relations)
    relations = top_down_semi_join(join_tree, post_order, relations)
    final_result = final_join_and_project(relations, join_tree, post_order)

    return final_result

# Apply final join and projection
start_time = time.time()
final_result = execute_operation(R1, R2, R3)
end_time = time.time()
print("Problem 2 final set:", final_result)
print("Problem 2 answer length:", len(final_result))
print("Problem 2 running time:", end_time - start_time)

Problem 2 final set: {(31, 3202, 82, 82), (43, 637, 76, 76), (89, 3895, 34, 34)}
Problem 2 answer length: 3
Problem 2 running time: 0.0001876354217529297


Implementing Problem 3 Algorithm with above generated dataset

In [102]:
# Perform the joins
def run_sequential_joins_4(R1, R2, R3):
  R1_2 = hash_join(R1, R2, 1, 0)  # Join R1 and R2 on R1.x and R2.y
  R1_3 = hash_join(R1_2, R3, 2, 0)  # Final join with R3 on R1_2.j and R3.l
  return R1_3

import time
# Timing and running Problem 3's solution
start_time = time.time()
results_problem3 = run_sequential_joins_4(R1, R2, R3)
end_time = time.time()
print("Problem 3 results:", results_problem3)
print("Problem 3 running time:", end_time - start_time)
print("Problem 3 output lentgth: ", len(results_problem3))  # Show the resulting tuples and count how many results were obtained

Problem 3 results: [(31, 3202, 82, 82), (43, 637, 76, 76), (89, 3895, 34, 34)]
Problem 3 running time: 0.0004050731658935547
Problem 3 output lentgth:  3


# Problem 5

Creating the Dataset

In [103]:
# Generate the dataset as specified
def generate_dataset_problem_5():
    # Relation R1
    R1 = [(i, 5) for i in range(1, 1001)] + [(i, 7) for i in range(1001, 2001)]
    R1.append((2001, 2002))
    random.shuffle(R1)

    # Relation R2
    R2 = [(5, j) for j in range(1, 1001)] + [(7, i) for i in range(1001, 2001)]
    R2.append((2002, 8))
    random.shuffle(R2)

    # Relation R3
    R3 = [(random.randint(2002, 3000), random.randint(1, 3000)) for _ in range(2000)]
    R3.append((8, 30))
    random.shuffle(R3)

    return R1, R2, R3


In [104]:

R1, R2, R3 = generate_dataset_problem_5()
print("R1 Dataset: ",R1)
print("R2 Dataset: ",R2)
print("R3 Dataset: ",R3)

R1 Dataset:  [(217, 5), (437, 5), (49, 5), (125, 5), (1993, 7), (225, 5), (201, 5), (1832, 7), (1324, 7), (1124, 7), (1490, 7), (899, 5), (237, 5), (661, 5), (1799, 7), (309, 5), (1553, 7), (1225, 7), (239, 5), (1605, 7), (1346, 7), (37, 5), (1132, 7), (994, 5), (420, 5), (1992, 7), (279, 5), (96, 5), (1577, 7), (1749, 7), (355, 5), (997, 5), (402, 5), (1451, 7), (881, 5), (1460, 7), (1914, 7), (390, 5), (672, 5), (923, 5), (768, 5), (667, 5), (1586, 7), (1944, 7), (515, 5), (1220, 7), (169, 5), (916, 5), (284, 5), (828, 5), (1568, 7), (84, 5), (663, 5), (1847, 7), (767, 5), (688, 5), (1883, 7), (1073, 7), (275, 5), (469, 5), (373, 5), (342, 5), (1761, 7), (1591, 7), (349, 5), (118, 5), (1420, 7), (1806, 7), (1472, 7), (1710, 7), (553, 5), (533, 5), (1371, 7), (111, 5), (1957, 7), (1864, 7), (1911, 7), (1522, 7), (1867, 7), (814, 5), (563, 5), (400, 5), (1377, 7), (902, 5), (1093, 7), (1550, 7), (1442, 7), (979, 5), (1141, 7), (46, 5), (254, 5), (855, 5), (1406, 7), (815, 5), (1649, 7)

Implementing Problem-2 Algorithm with above generated dataset

In [108]:
# Apply final join and projection
start_time = time.time()
final_result = execute_operation(R1, R2, R3)
end_time = time.time()
print("Problem 2 final set:", final_result)
print("Problem 2 running time:", end_time - start_time)
print("Problem 2 answer length:", len(final_result))

Problem 2 final set: {(868, 5, 8, 30), (70, 5, 8, 30), (636, 5, 8, 30), (929, 5, 8, 30), (926, 5, 8, 30), (161, 5, 8, 30), (408, 5, 8, 30), (77, 5, 8, 30), (464, 5, 8, 30), (792, 5, 8, 30), (572, 5, 8, 30), (347, 5, 8, 30), (865, 5, 8, 30), (654, 5, 8, 30), (39, 5, 8, 30), (557, 5, 8, 30), (947, 5, 8, 30), (540, 5, 8, 30), (944, 5, 8, 30), (332, 5, 8, 30), (329, 5, 8, 30), (315, 5, 8, 30), (312, 5, 8, 30), (440, 5, 8, 30), (847, 5, 8, 30), (850, 5, 8, 30), (388, 5, 8, 30), (833, 5, 8, 30), (80, 5, 8, 30), (420, 5, 8, 30), (723, 5, 8, 30), (476, 5, 8, 30), (251, 5, 8, 30), (461, 5, 8, 30), (377, 5, 8, 30), (16, 5, 8, 30), (764, 5, 8, 30), (895, 5, 8, 30), (670, 5, 8, 30), (667, 5, 8, 30), (875, 5, 8, 30), (872, 5, 8, 30), (354, 5, 8, 30), (778, 5, 8, 30), (775, 5, 8, 30), (257, 5, 8, 30), (647, 5, 8, 30), (316, 5, 8, 30), (313, 5, 8, 30), (857, 5, 8, 30), (88, 5, 8, 30), (398, 5, 8, 30), (395, 5, 8, 30), (606, 5, 8, 30), (735, 5, 8, 30), (298, 5, 8, 30), (295, 5, 8, 30), (714, 5, 8, 30)

Implementing Problem-3 Algorithm with above generated dataset

In [107]:
# Perform the joins
def run_sequential_joins_5(R1, R2, R3):
  R1_2 = hash_join(R1, R2, 1, 0)  # Join R1 and R2 on R1.x and R2.y
  R1_3 = hash_join(R1_2, R3, 2, 0)  # Final join with R3 on R1_2.j and R3.l
  return R1_3

import time
# Timing and running Problem 3's solution
start_time = time.time()
results_problem3 = run_sequential_joins_5(R1, R2, R3)
end_time = time.time()
print("Problem 3 results:", results_problem3)
print("Problem 3 running time:", end_time - start_time)
print("Problem 2 answer length:", len(results_problem3))

Problem 3 results: [(217, 5, 8, 30), (437, 5, 8, 30), (49, 5, 8, 30), (125, 5, 8, 30), (225, 5, 8, 30), (201, 5, 8, 30), (899, 5, 8, 30), (237, 5, 8, 30), (661, 5, 8, 30), (309, 5, 8, 30), (239, 5, 8, 30), (37, 5, 8, 30), (994, 5, 8, 30), (420, 5, 8, 30), (279, 5, 8, 30), (96, 5, 8, 30), (355, 5, 8, 30), (997, 5, 8, 30), (402, 5, 8, 30), (881, 5, 8, 30), (390, 5, 8, 30), (672, 5, 8, 30), (923, 5, 8, 30), (768, 5, 8, 30), (667, 5, 8, 30), (515, 5, 8, 30), (169, 5, 8, 30), (916, 5, 8, 30), (284, 5, 8, 30), (828, 5, 8, 30), (84, 5, 8, 30), (663, 5, 8, 30), (767, 5, 8, 30), (688, 5, 8, 30), (275, 5, 8, 30), (469, 5, 8, 30), (373, 5, 8, 30), (342, 5, 8, 30), (349, 5, 8, 30), (118, 5, 8, 30), (553, 5, 8, 30), (533, 5, 8, 30), (111, 5, 8, 30), (814, 5, 8, 30), (563, 5, 8, 30), (400, 5, 8, 30), (902, 5, 8, 30), (979, 5, 8, 30), (46, 5, 8, 30), (254, 5, 8, 30), (855, 5, 8, 30), (815, 5, 8, 30), (848, 5, 8, 30), (987, 5, 8, 30), (42, 5, 8, 30), (634, 5, 8, 30), (873, 5, 8, 30), (92, 5, 8, 30), (

# Downloading Problem 5 Dataset for Problem 6

In [39]:
def generate_and_save_dataset_direct(R1,R2,R3):
    try:
        # Write R1 to a CSV file
        with open('/content/R1.csv', 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['A', 'B'])
            writer.writerows(R1)

        # Write R2 to a CSV file
        with open('/content/R2.csv', 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['B', 'C'])
            writer.writerows(R2)

        # Write R3 to a CSV file
        with open('/content/R3.csv', 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['C', 'D'])
            writer.writerows(R3)

    except KeyError as e:
        print(f"KeyError encountered: {e}")
        return False
    return True

# Execute the simplified dataset generation and saving
save = generate_and_save_dataset_direct(R1,R2,R3)



# SQL code for Problem 6

In [ ]:
SELECT R1.A, R1.B, R2.C, R3.D
FROM R1
JOIN R2 ON R1.B = R2.B
JOIN R3 ON R2.C = R3.C;